In [ ]:
!pip install datasets
!pip install transformers>=4.37.0
!pip datasets torchvision accelerate
!pip install --upgrade sympy
!pip install tiktoken transformers_stream_generator timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
ERROR: unknown command "datasets"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing i

In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#LOADING DATASET
import pandas as pd
# Path to your zip file
zip_path = '/content/drive/MyDrive/instagram_data.zip'

# Unzipping the dataset
output_dir = '/content/dataset'
os.makedirs(output_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

In [ ]:
#LOAD CAPTIONS
# List images and captions
image_dir = os.path.join(output_dir, 'img')
caption_file = '/content/drive/MyDrive/actual_captions_csv.csv'

# Load captions into a DataFrame
captions_df = pd.read_csv(caption_file, delimiter=',', header=None, names=['id', 'image_path', 'caption'])
captions_df = captions_df.head(1000)

# Display the loaded captions DataFrame
print(captions_df)

       id     image_path                                            caption
0       1     img/insta1                                                NaN
1       2     img/insta2                                               bye 
2       3     img/insta3  Ok, a few more... sorry I just had so much fun...
3       4     img/insta4  This was one of my favorite shoots I’ve ever d...
4       5     img/insta5               Wrapped round my finger like a ring 
..    ...            ...                                                ...
995   996   img/insta996                                           preppin 
996   997   img/insta997                                                .. 
997   998   img/insta998                                        MET 2014.. 
998   999   img/insta999                             Paris Photo Paramount 
999  1000  img/insta1000                  Them is EVERYTHING tho 🙌 allofme 

[1000 rows x 3 columns]


In [ ]:
#MOOD LABELS
# CLIP Model for Mood and Aesthetic Labeling
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os

# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

# Define three categories of prompts
color_prompts = ["pastel colors", "monochrome", "black and white", "cool colors", "vibrant"]
mood_prompts = ["joyful", "melancholic", "romantic", "abstract", "cyberpunk", "minimalistic", "retro"]

# Preprocess and encode text prompts for each category
color_text_inputs = clip_processor(text=color_prompts, return_tensors="pt", padding=True)
mood_text_inputs = clip_processor(text=mood_prompts, return_tensors="pt", padding=True)

def get_mood_labels(image_path):
    img = Image.open(image_path).convert("RGB")
    image_inputs = clip_processor(images=img, return_tensors="pt")

    with torch.no_grad():
        # Get CLIP embeddings for the image
        image_features = clip_model.get_image_features(**image_inputs)

        # Get text embeddings and cosine similarity for each category
        color_features = clip_model.get_text_features(**color_text_inputs)
        color_similarity = torch.nn.functional.cosine_similarity(image_features, color_features)
        mood_features = clip_model.get_text_features(**mood_text_inputs)
        mood_similarity = torch.nn.functional.cosine_similarity(image_features, mood_features)

        # Get top 3 labels from Color-Based, top 2 from Mood-Based
        top_color_indices = torch.argsort(color_similarity, descending=True)[:2]
        top_mood_indices = torch.argsort(mood_similarity, descending=True)[:2]

        # Combine all top labels into a single list
        labels = [color_prompts[i] for i in top_color_indices] + \
                 [mood_prompts[i] for i in top_mood_indices]

    return labels

# Separate dictionary for mood and aesthetic labels
mood_aesthetic_labels = {}
for _, row in captions_df.iterrows():
    image_path = os.path.join(output_dir, row['image_path'] + '.jpg')
    mood_aesthetic_labels[row['image_path']] = get_mood_labels(image_path)

# Display mood and aesthetic labels for each image
for image_name, labels in mood_aesthetic_labels.items():
    print(f"Image: {image_name}")
    print("Labels:", labels)
    print("\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Image: img/insta1
Labels: ['vibrant', 'black and white', 'retro', 'joyful']


Image: img/insta2
Labels: ['cool colors', 'pastel colors', 'retro', 'melancholic']


Image: img/insta3
Labels: ['pastel colors', 'vibrant', 'joyful', 'melancholic']


Image: img/insta4
Labels: ['pastel colors', 'vibrant', 'retro', 'abstract']


Image: img/insta5
Labels: ['pastel colors', 'black and white', 'romantic', 'melancholic']


Image: img/insta6
Labels: ['black and white', 'vibrant', 'joyful', 'romantic']


Image: img/insta7
Labels: ['vibrant', 'cool colors', 'retro', 'joyful']


Image: img/insta8
Labels: ['vibrant', 'black and white', 'romantic', 'joyful']


Image: img/insta9
Labels: ['black and white', 'monochrome', 'joyful', 'romantic']


Image: img/insta10
Labels: ['pastel colors', 'cool colors', 'joyful', 'retro']


Image: img/insta11
Labels: ['black and white', 'vibrant', 'retro', 'melancholic']


Image: img/insta12
Labels: ['pastel colors', 'vibrant', 'joyful', 'retro']


Image: img/insta13
Labe

In [ ]:
#IMAGE DESCRIPTION
# Image Captioning with BLIP
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import torch

# Load BLIP model and processor
model_name = "Salesforce/blip-image-captioning-base"
blip_model = BlipForConditionalGeneration.from_pretrained(model_name)
blip_processor = BlipProcessor.from_pretrained(model_name)

def generate_image_description(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = blip_processor(images=img, return_tensors="pt")

    # Generate caption
    with torch.no_grad():
        output = blip_model.generate(**inputs)

    # Decode the output to get the caption text
    description = blip_processor.decode(output[0], skip_special_tokens=True)
    return description

# Separate dictionary for image descriptions
image_descriptions = {}
for _, row in captions_df.iterrows():
    image_path = os.path.join(output_dir, row['image_path'] + '.jpg')
    image_descriptions[row['image_path']] = generate_image_description(image_path)

# Display image descriptions for each image
for image_name, description in image_descriptions.items():
    print(f"Image: {image_name}")
    print("Description:", description)
    print("\n")


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Image: img/insta1
Description: a woman with wavy hair and a grey sweater


Image: img/insta2
Description: a woman in pink gloves posing on a rooftop


Image: img/insta3
Description: a woman wearing a white sweater and a white scarf


Image: img/insta4
Description: a woman with a large eye patch on her forehead


Image: img/insta5
Description: a woman getting her make up


Image: img/insta6
Description: a woman in a black dress walks down the runway


Image: img/insta7
Description: a woman sitting in the back seat of a car


Image: img/insta8
Description: a woman in a bikini top with the words semaz en no 1 am on it


Image: img/insta9
Description: a woman in a dress standing under a tree


Image: img/insta10
Description: a man in a white shirt


Image: img/insta11
Description: a woman with long hair and a black shirt


Image: img/insta12
Description: a woman in a pink sweater and black pants


Image: img/insta13
Description: a woman in a gold dress posing for a photo


Image: img/insta

In [ ]:
#SAVING BLIP DESCRIPTIONS IN GOOGLE DRIVE
import os
import json
# Specify the path where you want to save the file on Google Drive
drive_path = '/content/drive/MyDrive/image_descriptions.json'

# Save the image_descriptions dictionary to a JSON file in Google Drive
with open(drive_path, 'w') as f:
    json.dump(image_descriptions, f, indent=4)

print(f"Image descriptions saved to {drive_path}")

Image descriptions saved to /content/drive/MyDrive/image_descriptions.json


In [ ]:
#CAPTION PREPROCESSING:MASKING TAGS
import json
import re
import random

# Define regex patterns
tag_pattern = r"@\w+"

def mask_randomly(text, pattern, mask, prob):
    """Mask items in text based on probability."""
    items = re.findall(pattern, text)
    for item in items:
        if random.random() < prob:
            text = text.replace(item, mask, 1)
    return text

def preprocess_caption(caption):
    # Ensure the caption is a string before applying regex
    if not isinstance(caption, str):
        caption = str(caption)  # Convert to string if not already
    caption = re.sub(tag_pattern, '[TAG]', caption)
    return caption

# Apply preprocessing to each caption in the DataFrame
preprocessed_captions = {}
for _, row in captions_df.iterrows():
    image_path = row['image_path']
    caption = row['caption']

    # Ensure caption is not None or handle missing captions
    if caption is None:
        caption = ""  # You can set this to a default string or handle differently

    preprocessed_captions[image_path] = preprocess_caption(caption)

# Print the preprocessed captions
for image_path, preprocessed_caption in preprocessed_captions.items():
    print(f"Image: {image_path}")
    print(f"Preprocessed Caption: {preprocessed_caption}")
    print("\n")

Image: img/insta1
Preprocessed Caption: nan


Image: img/insta2
Preprocessed Caption: bye 


Image: img/insta3
Preprocessed Caption: Ok, a few more... sorry I just had so much fun that day 


Image: img/insta4
Preprocessed Caption: This was one of my favorite shoots I’ve ever done! Thank you -a seriously badass woman, for shooting my cover! And thank you to everyone who asked a question for the interview 🤍 


Image: img/insta5
Preprocessed Caption: Wrapped round my finger like a ring 


Image: img/insta6
Preprocessed Caption: Yesterday was so much fun!! has been such a huge figure in my life. I love her humor, her grace and her heart! Check out the interview where I’m gushing over her the whole time! 


Image: img/insta7
Preprocessed Caption: Well.. 


Image: img/insta8
Preprocessed Caption: It’s officially out! I was a bit embarrassed asking so often for you to stream or buy my album. It felt inauthentic. Thank YOU so much for making something so personal to me be a moment I’ll never 

In [ ]:
#COMBINED DATASET
# Combine data into a single dataset with four columns: image labels, mood and aesthetic labels, captions, and image descriptions
combined_dataset = []

# Process each image entry
for _, row in captions_df.iterrows():
    image_path = row['image_path']

    # Ensure all required labels and descriptions exist in the dictionaries
    if  image_path not in mood_aesthetic_labels or image_path not in image_descriptions or image_path not in preprocessed_captions:
        print(f"Skipping {image_path} due to missing data.")
        continue

    # Get preprocessed caption, image classification labels, mood/aesthetic labels, and image description
    preprocessed_caption = preprocessed_captions[image_path]
    mood_labels = mood_aesthetic_labels[image_path]
    image_description = image_descriptions[image_path]

    # Combine all information into a single entry
    combined_entry = {
        "mood_aesthetic_labels": mood_labels,
        "image_description": image_description,
        "preprocessed_caption": preprocessed_caption
    }
    combined_dataset.append(combined_entry)

# Convert combined dataset to DataFrame
combined_df = pd.DataFrame(combined_dataset)

# Save combined dataset to a CSV file
combined_dataset_path = '/content/drive/MyDrive/combined_training_dataset.csv'
combined_df.to_csv(combined_dataset_path, index=False)

print("Combined Dataset:")
print(combined_df.head())


Combined Dataset:
                               mood_aesthetic_labels  \
0          [vibrant, black and white, retro, joyful]   
1   [cool colors, pastel colors, retro, melancholic]   
2      [pastel colors, vibrant, joyful, melancholic]   
3          [pastel colors, vibrant, retro, abstract]   
4  [pastel colors, black and white, romantic, mel...   

                                   image_description  \
0          a woman with wavy hair and a grey sweater   
1         a woman in pink gloves posing on a rooftop   
2  a woman wearing a white sweater and a white scarf   
3     a woman with a large eye patch on her forehead   
4                        a woman getting her make up   

                                preprocessed_caption  
0                                                nan  
1                                               bye   
2  Ok, a few more... sorry I just had so much fun...  
3  This was one of my favorite shoots I’ve ever d...  
4               Wrapped round my 

In [ ]:
# COMBINING DATASET WITH MODEL PROMPT FOR FINE TUNING
new_dataset = []

# Process each entry in the combined dataset
for _, row in combined_df.iterrows():
    # Combine image description and mood/aesthetic labels into a single sentence
    image_description = row['image_description']
    mood_labels = row['mood_aesthetic_labels']

    # Format mood labels with hashtags and underscores
    formatted_mood_labels = " ".join([f"#{label.replace(' ', '_')}" for label in mood_labels])
    combined_description = ("You are an expert AI social-media caption generator. Based on the image's mood and description, "
        "generate a funky, vibey and Instagram-worthy caption. Keep it fun, short and "
        "attractive (max 50 characters).\n\n"
        "Mood Labels: " + formatted_mood_labels + "\n"
        "Image Description: " + image_description + "\n"
        "Caption:")

    # Get preprocessed caption
    preprocessed_caption = row['preprocessed_caption']

    # Append to the new dataset
    new_entry = {
        "description_and_labels": combined_description,
        "preprocessed_caption": preprocessed_caption
    }
    new_dataset.append(new_entry)

# Convert new dataset to DataFrame
new_df = pd.DataFrame(new_dataset)

# Save the new dataset to a CSV file
new_dataset_path = '/content/drive/MyDrive/new_training_dataset.csv'
new_df.to_csv(new_dataset_path, index=False)

print("New Dataset:")
print(new_df.head())


New Dataset:
                              description_and_labels  \
0  You are an expert AI social-media caption gene...   
1  You are an expert AI social-media caption gene...   
2  You are an expert AI social-media caption gene...   
3  You are an expert AI social-media caption gene...   
4  You are an expert AI social-media caption gene...   

                                preprocessed_caption  
0                                                nan  
1                                               bye   
2  Ok, a few more... sorry I just had so much fun...  
3  This was one of my favorite shoots I’ve ever d...  
4               Wrapped round my finger like a ring   


In [ ]:
!pip install ms-swift[all] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.6 MB/s

In [ ]:
!pip install mpi4py

  Using cached mpi4py-4.0.1.tar.gz (466 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.1-cp310-cp310-linux_x86_64.whl size=4266340 sha256=50755d80e0dcd810993bcf42164632abe0ccc2e0cd9acdc29881e899977259d8
  Stored in directory: /root/.cache/pip/wheels/3c/ca/13/13218a83854023ccec184e3af482f0f038b434aa32c19afee8
Successfully built mpi4py


In [ ]:
!pip install git+https://github.com/microsoft/deepspeed

  Cloning https://github.com/microsoft/deepspeed to /tmp/pip-req-build-6_k4hbk0
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/deepspeed /tmp/pip-req-build-6_k4hbk0
  Resolved https://github.com/microsoft/deepspeed to commit f57b1ef18af25f7e0450d92d5f163ee8c887372a
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.5+f57b1ef1-py3-none-any.whl size=1535429 sha256=5b2d8447218823790c70f31d052b873c9d9b9c5daf566cd9068998acc4b9e06b
  Stored in directory: /tmp/pip-ephem-wheel-cache-l4qqvqkp/wheels/c7/2a/60/6cd1be5e9093337b6243efab085ade35b787b5c4c1aaf99387
Successfully built deepspeed


In [ ]:
#FINE TUNING WITH MS-SWIFT
!swift sft --model_id_or_path "qwen/Qwen2-7B" --template_type "qwen" --dataset "/content/drive/MyDrive/new_training_dataset.csv"  --dataset_test_ratio "0.10" --lora_target_modules "ALL" --init_lora_weights "True" --learning_rate "1e-4" --gradient_accumulation_steps "16" --eval_steps "500" --save_steps "500" --quant_method "bnb" --quantization_bit 4 --eval_batch_size "1" --ref_model_type "qwen-1_8b"  --deepspeed default-zero2  --add_output_dir_suffix False --output_dir /content/output/qwen2-7b/v0-20241118-203603 --logging_dir /content/output/qwen2-7b/v0-20241118-203603/runs --ignore_args_error True --max_length 128

2024-11-19 13:36:06.885718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-19 13:36:06.906016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-19 13:36:06.912737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 13:36:06.927325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 13:36:08.272284: W tensorflow/compiler/tf2

In [ ]:
#SAVE FINE TUNED MODEL IN DRIVE
!cp -r /content/output/qwen2-7b/v0-20241118-203603 /content/drive/MyDrive/qwen_fine_tuned_model

In [ ]:
#EVALUATION CODE FOR FINE TUNED MODEL
!CUDA_VISIBLE_DEVICES=0 swift infer --ckpt_dir /content/drive/MyDrive/qwen_fine_tuned_model/checkpoint-48 --load_dataset_config true --merge_lora true --max_model_len 128 --top_k 50 --top_p 0.95 --temperature 0.7 --repetition_penalty 1.2

2024-11-22 04:17:45.197001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 04:17:45.230102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 04:17:45.239812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 04:17:45.261505: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 04:17:46.928874: W tensorflow/comp

In [ ]:
#TERMINAL INFERENCE CODE WITH DEBUGGING
import os
import json
import csv
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
import torch
from google.colab import files

# Load models
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

blip_model_name = "Salesforce/blip-image-captioning-base"
blip_model = BlipForConditionalGeneration.from_pretrained(blip_model_name)
blip_processor = BlipProcessor.from_pretrained(blip_model_name)

# Load fine-tuned model from Google Drive (Adjust the path as per your setup)
fine_tuned_model_path = "/content/drive/MyDrive/qwen_fine_tuned_model/checkpoint-48-merged"
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)

# Prompts for mood and aesthetic labels
color_prompts = ["pastel colors", "monochrome", "black and white", "cool colors", "vibrant"]
mood_prompts = ["joyful", "melancholic", "romantic", "abstract", "cyberpunk", "minimalistic", "retro"]
color_text_inputs = clip_processor(text=color_prompts, return_tensors="pt", padding=True)
mood_text_inputs = clip_processor(text=mood_prompts, return_tensors="pt", padding=True)

# Function to extract mood and aesthetic labels
def get_mood_labels(image_path):
    """Extract mood and aesthetic labels using CLIP."""
    img = Image.open(image_path).convert("RGB")
    image_inputs = clip_processor(images=img, return_tensors="pt")

    with torch.no_grad():
        image_features = clip_model.get_image_features(**image_inputs)
        color_features = clip_model.get_text_features(**color_text_inputs)
        mood_features = clip_model.get_text_features(**mood_text_inputs)

        # Calculate similarity
        color_similarity = torch.nn.functional.cosine_similarity(image_features, color_features)
        mood_similarity = torch.nn.functional.cosine_similarity(image_features, mood_features)

        # Top labels
        top_color_indices = torch.argsort(color_similarity, descending=True)[:2]
        top_mood_indices = torch.argsort(mood_similarity, descending=True)[:2]
        labels = [color_prompts[i] for i in top_color_indices] + \
                 [mood_prompts[i] for i in top_mood_indices]

    return labels

# Function to generate image description using BLIP
def generate_image_description(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = blip_processor(images=img, return_tensors="pt")

    with torch.no_grad():
        output = blip_model.generate(**inputs)
    description = blip_processor.decode(output[0], skip_special_tokens=True)
    return description

# File upload
print("Please upload your image file:")
uploaded = files.upload()

if not uploaded:
    print("Error: No file uploaded.")
    exit()

image_path = list(uploaded.keys())[0]

# Step 1: Extract mood labels
print("Extracting mood labels...")
mood_labels = get_mood_labels(image_path)
formatted_mood_labels = " ".join([f"#{label.replace(' ', '_')}" for label in mood_labels])
print("Mood Labels:", formatted_mood_labels)

# Step 2: Generate image description
print("Generating image description...")
image_description = generate_image_description(image_path)
print("Image Description:", image_description)

# Step 3: Construct the input for the model
instruction = (
    "You are an expert AI social-media caption generator. Based on the image's mood and description, "
    "generate a funky, vibey and Instagram-worthy caption. Keep it fun, short and attractive (max 50 characters)."
)
input_prompt = f"{instruction}\n\nMood Labels: {formatted_mood_labels}\nImage Description: {image_description}\nCaption:"

# Step 4: Inference using the fine-tuned model
inputs = tokenizer(input_prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(
        inputs["input_ids"],
        max_length=50,
        num_beams=5,
        top_p=0.92,
        top_k=50,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

generated_caption = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f"Generated Caption: {generated_caption}")

# Step 5: Save the input and output in CSV format (matching training data format)
input_csv_path = "/content/input.csv"
csv_columns = ["instruction", "output"]

# Create the CSV file with the required columns
with open(input_csv_path, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerow({"instruction": input_prompt, "output": generated_caption})  # Save the generated caption

# Debugging: Check if the CSV is correctly created
print(f"Input CSV saved at: {input_csv_path}")
with open(input_csv_path, "r") as f:
    print(f"Input CSV content: {f.read()}")


2024-11-23 16:53:38.817854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-23 16:53:38.837412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-23 16:53:38.843459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 16:53:38.859819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-23 16:53:40.339460: W tensorflow/comp

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

Saving pic_1.jpg to pic_1.jpg
Extracting mood labels...
Mood Labels: #pastel_colors #cool_colors #romantic #melancholic
Generating image description...


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.92` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Both `max_new_tokens` (=2048) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.c

Image Description: a man and woman sitting on the beach


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated Caption: You are an expert AI social-media caption generator. Based on the image's mood and description, generate a funky, vibey and Instagram-worthy caption. Keep it fun, short and attractive (max 50 characters).

Mood Labels: #pastel_colors #cool_colors #romantic #melancholic
Image Description: a man and woman sitting on the beach
Caption: "Sitting by the sea, watching the sunset with my soulmate."
Input CSV saved at: /content/input.csv
Input CSV content: instruction,output
"You are an expert AI social-media caption generator. Based on the image's mood and description, generate a funky, vibey and Instagram-worthy caption. Keep it fun, short and attractive (max 50 characters).

Mood Labels: #pastel_colors #cool_colors #romantic #melancholic
Image Description: a man and woman sitting on the beach
Caption:","You are an expert AI social-media caption generator. Based on the image's mood and description, generate a funky, vibey and Instagram-worthy caption. Keep it fun, short an

In [ ]:
!pip install streamlit requests
!pip install streamlit -q
!pip install pyngrok
!pip install -U bitsandbytes

In [ ]:
#INFERENCE CODE WITH STREAMLIT FRONTEND FINAL
%%writefile app.py
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
import torch
import streamlit as st

# Set page configuration with blue theme
st.set_page_config(
    page_title="Captify ✨📷",
    page_icon="📷",
    layout="centered",
    initial_sidebar_state="collapsed"
)

# Load models
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

blip_model_name = "Salesforce/blip-image-captioning-base"
blip_model = BlipForConditionalGeneration.from_pretrained(blip_model_name)
blip_processor = BlipProcessor.from_pretrained(blip_model_name)

fine_tuned_model_path = "/content/drive/MyDrive/qwen_fine_tuned_model/checkpoint-48-merged"
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)

# Prompts for mood and aesthetic labels
color_prompts = ["pastel colors", "monochrome", "black and white", "cool colors", "vibrant"]
mood_prompts = ["joyful", "melancholic", "romantic", "abstract", "cyberpunk", "minimalistic", "retro"]
color_text_inputs = clip_processor(text=color_prompts, return_tensors="pt", padding=True)
mood_text_inputs = clip_processor(text=mood_prompts, return_tensors="pt", padding=True)

# Function to extract mood and aesthetic labels
def get_mood_labels(image_path):
    img = Image.open(image_path).convert("RGB")
    image_inputs = clip_processor(images=img, return_tensors="pt")
    with torch.no_grad():
        image_features = clip_model.get_image_features(**image_inputs)
        color_features = clip_model.get_text_features(**color_text_inputs)
        mood_features = clip_model.get_text_features(**mood_text_inputs)
        color_similarity = torch.nn.functional.cosine_similarity(image_features, color_features)
        mood_similarity = torch.nn.functional.cosine_similarity(image_features, mood_features)
        top_color_indices = torch.argsort(color_similarity, descending=True)[:2]
        top_mood_indices = torch.argsort(mood_similarity, descending=True)[:2]
        labels = [color_prompts[i] for i in top_color_indices] + [mood_prompts[i] for i in top_mood_indices]
    return labels

# Function to generate image description using BLIP
def generate_image_description(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = blip_processor(images=img, return_tensors="pt")
    with torch.no_grad():
        output = blip_model.generate(**inputs)
    description = blip_processor.decode(output[0], skip_special_tokens=True)
    return description

# Add logo at the top
logo_path = "logo.png"
st.markdown(
    """
    <style>
        body {
            background-color: #eaf6ff; /* Light blue theme */
        }
        .title {
            font-size: 3rem;
            color: #0056b3; /* Blue color for the title */
            font-weight: bold;
            text-align: center;
        }
    </style>
    """,
    unsafe_allow_html=True
)

# Display project logo
if os.path.exists(logo_path):
    st.image(logo_path, use_column_width=True)

# App title with sparkle and camera emoji
st.markdown('<h1 class="title">✨📷 Captify 📷✨</h1>', unsafe_allow_html=True)

# Description below the title
st.markdown(
    "<h3 style='text-align: center; color: #0056b3;'>Transform your images into stunning captions!</h3>",
    unsafe_allow_html=True
)

# Image upload
uploaded_file = st.file_uploader("Upload an Image", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # Display the uploaded image with rounded corners
    st.markdown("<h3 style='text-align: center; color: #0056b3;'>Uploaded Image:</h3>", unsafe_allow_html=True)
    st.image(uploaded_file, caption="Your Image", use_column_width=True)

    # Save the uploaded file temporarily to process
    image_path = "/tmp/uploaded_image.jpg"
    with open(image_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Extract mood labels
    mood_labels = get_mood_labels(image_path)
    formatted_mood_labels = " ".join([f"#{label.replace(' ', '_')}" for label in mood_labels])
    st.markdown(
        f"<h4 style='text-align: center; color: #0056b3;'>Mood Labels: {formatted_mood_labels}</h4>",
        unsafe_allow_html=True
    )

    # Generate image description
    image_description = generate_image_description(image_path)
    st.markdown(
        f"<h4 style='text-align: center; color: #0056b3;'>Image Description: {image_description}</h4>",
        unsafe_allow_html=True
    )

    # Construct the input for the model
    instruction = (
        "You are an expert AI social-media caption generator. Based on the image's mood and description, "
        "generate a funky, vibey and Instagram-worthy caption. Keep it fun, short and attractive (max 50 characters)."
    )
    input_prompt = f"{instruction}\n\nMood Labels: {formatted_mood_labels}\nImage Description: {image_description}\nCaption:"

    # Inference using the fine-tuned model
     # Inference using the fine-tuned model
    inputs = tokenizer(input_prompt, return_tensors="pt")
    with torch.no_grad():
        generated_ids = model.generate(
            inputs["input_ids"],
            max_length=50,
            num_beams=5,
            top_p=0.92,
            top_k=50,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True
        )

    # Extract the caption only
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    if "Caption:" in generated_text:
        caption = generated_text.split("Caption:")[-1].strip()
    else:
        caption = generated_text  # Fallback

    # Display the generated caption
    st.markdown(
        f"<h4 style='text-align: center; color: #0056b3;'>{caption}</h4>",
        unsafe_allow_html=True
    )

Writing app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2pGHJaX4ACJPgJTycO8Kn4k8NSm_6V9hyRmp1JoqUeSCb8AE7")

# Start Streamlit server on a specific port
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://3dcc-34-19-22-247.ngrok-free.app
